In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random

from tqdm import tqdm

from sqlalchemy import create_engine
import psycopg2 as engine

dates=pd.date_range(start='1/1/2018', periods=784, freq='B') # generating date time range no weekends
print(f'How long is my data: {len(dates)}')

cities=pd.read_csv('resources/worldcities.csv')
cities.head()

sales = {
    "date":[],   # decided to add much later
    "country":[],
    "city":[],
    "lat":[],
    "lng":[],
    "product_type":[],
    "product":[],
    "qty": [],
    "price":[],
    "price_wd":[],
    "discount":[]
} # empty dir to hold sales data


regions=['United States','Mexico','Canada']
product_types=['P&C','Benefits']
products=['A','B','C']
discount=[True,False]
price_A=np.random.normal(loc=1000,scale=0.5,size=100)
price_B=np.random.normal(loc=1200,scale=1,size=100)
price_C=np.random.normal(loc=1500,scale=2,size=100)


def pickItem(list_items):
    return random.choice(list_items)



# %%timeit
for i in tqdm(range(0,len(dates))):
    for j in range(random.randint(1,5)):
        city_c=cities[['country','city','lat','lng']].sample().iloc[0]
        sales['date'].append(dates[i])
        sales['country'].append(city_c[0])
        sales['city'].append(city_c[1])
        sales['lat'].append(city_c[2])
        sales['lng'].append(city_c[3])
        sales['product_type'].append(pickItem(product_types))
        sales['qty'].append(random.randint(1,10))  
        product=pickItem(products)
        sales['product'].append(product)
        if product=='A':
            sales['price'].append(round(random.choice(price_A),2))
        elif product=='B':
            sales['price'].append(round(random.choice(price_C)))
        else:
            sales['price'].append(round(random.choice(price_C)))
        discount_choice=random.choice(discount)
        if discount_choice==True:
            sales['discount'].append(True)
            sales['price_wd'].append(.95)
        else:
            sales['discount'].append(False)
            sales['price_wd'].append(1)
# sales['date']=dates  # i can always add the time component to the index later


sales_df=pd.DataFrame(sales)
sales_df['Revenue']=sales_df['qty']*sales_df['price']*sales_df['price_wd']
sales_df.set_index(sales_df['date'],inplace=True)
# sales_df.index.name='dates'
sales_df['month']=sales_df.index.month
sales_df['week']=sales_df.index.week
sales_df['year']=sales_df.index.year
sales_df['dow']=sales_df.index.dayofweek +1
sales_df.head()

sales_df.groupby(['city','country','lat','lng']).agg({'Revenue':'sum','qty':'mean'})
sales_df_by_dow=sales_df.groupby(['city','country','lat','lng']).agg({'Revenue':'sum','qty':'mean'})
sales_df_by_dow.to_csv('abc_by_city.csv')

How long is my data: 784


100%|██████████| 784/784 [00:06<00:00, 123.47it/s]


In [121]:
sales_df

,date,country,city,lat,lng,product_type,product,qty,price,price_wd,discount,Revenue,month,week,year,dow
date,,,,,,,,,,,,,,,,
2018-01-01,2018-01-01,United States,Allegan,42.5299,-85.8462,P&C,A,7,999.97,1.00,False,6999.79,1,1,2018,1
2018-01-01,2018-01-01,United States,Seal Beach,33.7542,-118.0714,Benefits,C,8,1501.00,0.95,True,11407.60,1,1,2018,1
2018-01-01,2018-01-01,United States,East Brunswick,40.4281,-74.4179,Benefits,B,4,1500.00,1.00,False,6000.00,1,1,2018,1
2018-01-01,2018-01-01,United States,Palm Springs North,25.9351,-80.3339,P&C,A,10,999.74,0.95,True,9497.53,1,1,2018,1
2018-01-01,2018-01-01,Guyana,Bartica,6.4104,-58.6300,P&C,B,1,1502.00,0.95,True,1426.90,1,1,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30,2020-12-30,Côte D’Ivoire,Sassandra,4.9504,-6.0833,Benefits,A,3,999.83,1.00,False,2999.49,12,53,2020,3
2020-12-30,2020-12-30,Nigeria,Ijebu-Ode,6.8204,3.9200,P&C,A,6,999.88,1.00,False,5999.28,12,53,2020,3
2020-12-30,2020-12-30,United States,Clermont,28.5403,-81.7296,Benefits,A,1,999.88,1.00,False,999.88,12,53,2020,3


In [91]:
df_rev2018 = sales_df.loc['2018-01-01':'2018-12-31']
df_rev2019 = sales_df.loc['2019-01-01':'2019-12-31']
df_rev2020 = sales_df.loc['2020-01-01':'2020-12-31']

In [ ]:
#some testsing above ...

In [3]:
df = sales_df.copy()

In [4]:
df = df.drop(['date'], axis=1)

In [5]:
df.reset_index(inplace= True)

In [6]:
df = df.rename(columns={'Revenue':'revenue'})

In [7]:
df

,date,country,city,lat,lng,product_type,product,qty,price,price_wd,discount,revenue,month,week,year,dow
0,2018-01-01,United States,Allegan,42.5299,-85.8462,P&C,A,7,999.97,1.00,False,6999.79,1,1,2018,1
1,2018-01-01,United States,Seal Beach,33.7542,-118.0714,Benefits,C,8,1501.00,0.95,True,11407.60,1,1,2018,1
2,2018-01-01,United States,East Brunswick,40.4281,-74.4179,Benefits,B,4,1500.00,1.00,False,6000.00,1,1,2018,1
3,2018-01-01,United States,Palm Springs North,25.9351,-80.3339,P&C,A,10,999.74,0.95,True,9497.53,1,1,2018,1
4,2018-01-01,Guyana,Bartica,6.4104,-58.6300,P&C,B,1,1502.00,0.95,True,1426.90,1,1,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,2020-12-30,Côte D’Ivoire,Sassandra,4.9504,-6.0833,Benefits,A,3,999.83,1.00,False,2999.49,12,53,2020,3
2381,2020-12-30,Nigeria,Ijebu-Ode,6.8204,3.9200,P&C,A,6,999.88,1.00,False,5999.28,12,53,2020,3
2382,2020-12-30,United States,Clermont,28.5403,-81.7296,Benefits,A,1,999.88,1.00,False,999.88,12,53,2020,3
2383,2020-12-31,United States,Thornton,39.9205,-104.9443,P&C,C,8,1498.00,1.00,False,11984.00,12,53,2020,4


In [8]:
# df = np.round(df, decimals=2)

In [17]:
df

,date,country,city,lat,lng,product_type,product,qty,price,price_wd,discount,revenue,month,week,year,dow
0,2018-01-01,United States,Allegan,42.5299,-85.8462,P&C,A,7,999.97,1.00,False,6999.79,1,1,2018,1
1,2018-01-01,United States,Seal Beach,33.7542,-118.0714,Benefits,C,8,1501.00,0.95,True,11407.60,1,1,2018,1
2,2018-01-01,United States,East Brunswick,40.4281,-74.4179,Benefits,B,4,1500.00,1.00,False,6000.00,1,1,2018,1
3,2018-01-01,United States,Palm Springs North,25.9351,-80.3339,P&C,A,10,999.74,0.95,True,9497.53,1,1,2018,1
4,2018-01-01,Guyana,Bartica,6.4104,-58.6300,P&C,B,1,1502.00,0.95,True,1426.90,1,1,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,2020-12-30,Côte D’Ivoire,Sassandra,4.9504,-6.0833,Benefits,A,3,999.83,1.00,False,2999.49,12,53,2020,3
2381,2020-12-30,Nigeria,Ijebu-Ode,6.8204,3.9200,P&C,A,6,999.88,1.00,False,5999.28,12,53,2020,3
2382,2020-12-30,United States,Clermont,28.5403,-81.7296,Benefits,A,1,999.88,1.00,False,999.88,12,53,2020,3
2383,2020-12-31,United States,Thornton,39.9205,-104.9443,P&C,C,8,1498.00,1.00,False,11984.00,12,53,2020,4


In [122]:
connection_string = "postgres:Password123@127.0.0.1:5432/project2"
# connection_string = "postgres:LEytq7KJJ5A7A9CB@35.237.9.208:5432/project2"
engine = create_engine(f'postgresql://{connection_string}')

In [123]:
# Confirm tables
engine.table_names()

['sales']

In [16]:
df.to_sql(name='sales', con=engine, if_exists='append', index=False)